In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from your module
from animal_shelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "aacuserPW"
db = AnimalShelter(username, password)

df_all = pd.DataFrame.from_records(db.retrieve_all())
df_all.drop(columns=['_id'], inplace=True)

df_water_filtered = pd.DataFrame.from_records(db.retrieve_water_filtered())
df_water_filtered.drop(columns=['_id'], inplace=True)

df_mountain_filtered = pd.DataFrame.from_records(db.retrieve_mountain_filtered())
df_mountain_filtered.drop(columns=['_id'], inplace=True)

df_disaster_filtered = pd.DataFrame.from_records(db.retrieve_disaster_filtered())
df_disaster_filtered.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash(__name__)

image_filename = 'my-image.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Kyle Riggenbach'))),
    html.h4('unique identifier for Kyle Riggenbach'),
    html.Img(src=f"data:image/png;base64,{encoded_image.decode()}", style={'width': '300px', 'height': 'auto'}),
    html.A('Website', href='www.snhu.edu'),
    html.Hr(),
    html.Div( 
        dcc.RadioItems(
            id='filter-radio',
            options=[
                {'label': 'All Animals', 'value': 'all'},
                {'label': 'Filtered Animals', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'}
            ],
            value='all',
            labelStyle={'display': 'inline-block'}
        ),
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_all.columns],
        data=df_all.to_dict('records'),
        page_action='native',  # Enables pagination
        page_current=0,         # Starting page
        page_size=10,           # Number of rows per page
        style_table={'overflowX': 'auto'},  # Enable horizontal scroll
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])

# Interaction Between Components / Controller

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-radio', 'value')]
)
def update_table_data(filter_option):
    if filter_option == 'water':
        df = pd.DataFrame.from_records(db.retrieve_water_filtered())
    elif filter_option == 'mountain':
        df = pd.DataFrame.from_records(db.retrieve_mountain_filtered())
    elif filter_option == 'disaster':
        df = pd.DataFrame.from_records(db.retrieve_disaster_filtered())
    else:
        df = pd.DataFrame.from_records(db.retrieve_all())
    
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='outcome_type', title='Outcome Types')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):  
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])
                    ])
                ]
            )
        ])
    ]

app.run_server(debug=True)
